# 19. 동적 속성과 포로퍼티

속성 : 데이터 속성, 메서드, 프로퍼티
메서드 : 호출할 수 있는 속성
프로퍼티 >> 클래스 인터페이스 변경x 하고 공개 데이터 속성을 접근자 메서드로 대체 가능
점 표기법으로 표현된 속성에 대한 접근 __getattr__(), __setattr__() 등 특별 메서드를 호출해 평가함.

## 19.1. 동적 속성을 이용한 데이터 랭글링

### 19.1.1. 동적 속성을 이용해 JSON과 유사한 데이터 둘러보기
feed['Schedule']['events'][40]['name'] 
>> (자바에서) feed.Schedule.events[40].name
>> (파이썬) 비슷하게 작동하는 사용자 정의 dict클래스 구현

__getattr__() : 속성을 가져오기 위한 일반적인 과정 실패할 때만 인터프리터에서 호출함.

예 19-5) __getattr__() : self.__ data 딕셔너리에 그 이름의 속성이 존재하는지 확인. 이 때 FroaenJSON은 self.__ data.items()에 위임해 딕셔너리 메서드 처리할 수 있게 함. self.__ data에 속성 없으면 __getattr__()이 해당 이름 키로 사용해 self.__ data에서 항목 가져와 FrozenJSON.build()에 전달. 내포된 데이터 구조체 >> build() >> 내포된 FrozenJSON 객체 >> JSON 데이터에 내포된 구조체 순회 가능

### 19.1.2. 잘못된 속성명 문제

FrozenJSON.__init__() 안의 키가 파이썬 키워드이면 >> 뒤에 _ 붙이기

파이썬 str 클래스의 s.isidentifier() : s가 정당한 파이썬 식별자인지 판단
but 정당한 식별자가 아닌 문자열을 정당한 속성명으로 바꾸는 것 어려움.

### 19.1.3. __new__()를 이용한 융통성 있는 객체 생성

__init__() ; 생성자 메서드..라 부름..
__new__() : 실제로 객체를 생성하는 특별 메서드, 클래스 메서드로 반드시 객체 반환함. 반환된 객체가 __init__()의 인수 self로 전달
__init__() : 호출 시 객체 받고, 아무것도 반환x >> '초기화 메서드'임

__new__()는 다른 클래스의 객체도 반환할 수 있음. 이때는 __init__() 호출 x

### 19.1.4. shelve를 이용해 OSCON 피드 구조 변경하기

pickle : 파이썬 객체 직렬화 포맷, 객체를 포맷으로(부터) 변환하는 모듈
shelve가 pickle 보관

shelve.open()의 고위함수 : shelve.Shelf 객체 반환.

shelve.Shelf : dbm 모듈을 이용해 키-값 객체 보관하는 객체
  abc.MutableMapping 클래스 상속 >> 매핑형이 제공하는 핵심 메서드 제공
  sync(), close() 등 입출력 관리하는 메서드 제공. 콘텍스트 관리자이기도 함
  새로운 값이 키에 할당될 때마다 키와 값이 저장됨
  키는 반드시 문자열이어야
  값은 반드시 pickle 모듈이 처리할 수 있는 객체여야

__slots__ 속성이 클래스에 선언되어 있지 않으면 객체의 __dict__ 에 속성 들어있음 >> __dict__ 를 직접 매핑형으로 설정 >> 객체의 속성 묶음을 빠르게 정의 가능

### 19.1.5. 프로퍼티를 이용해 연결된 레코드 읽기

Record 클래스 : __init__() 메서드는 (예 19-9)와 동일. __eq__() 메서드 추가
DbRecord 클래스 : Record의 서브클래스. __ db 클래스 속성, 속성을 설정하고 가져오는 set_db(), get_db() 정적 메서드, 데이터베이스에서 레코드 가져오는 fetch() 클래스 메서드. 디버깅과 테스트 지원위한 __repr__() 객체 메서드 추가
Event 클래스 : DbRecord의 서브클래스, 연결된 레코드를 가져오기 위한 venue, speakers 프로퍼티, 특별메서드 __repr__() 추가

## 19.2. 속성을 검증하기 위한 프로퍼티 사용하기

### 19.2.1. LineItem 버전1) 주문항목 클래스

주문 수량 음수 가능..
>> LineItem의 인터페이스 변경해 weight 속성에 게터와 세터 사용.
 but,  상품의 무게에 직접 값을 할당해 설정하는게 자연스러움. 다른 부분에서 item.weight 속성에 접근하고 있을 수도 >> 데이터 속성 프로퍼티로
 
### 19.2.2. 버전2) 검증하는 프로퍼티

프로퍼티 구현 >> 게터, 세터 메서드 사용가능. but  LineItem의 인터페이스 바뀌지 않음

## 19.3. 프로퍼티 제대로 알아보기

내장 property() 데커레이터로 사용되는 경우 많음. but 사실 클래스임.

함수, 클래스 교환 가능; 둘 다 콜러블, new 연산자 없음 >> 생성자 호출, 팩토리 함수 호출 차이 없음. 
  if 장식된 함수 대체할 수 있는 콜러블 생성>> 둘 다 데커레이터로 사용가능

property(fget=None, fset=None, fedl=None, doc=None)
   모든 인수는 선택적. 인수에 함수 제공 x >> 프로퍼티 객체가 해당 연산 지원x

### 19.3.1. 객체 속성을 가리는 프로퍼티

프로퍼티 : 클래스 속성. but 실제로 클래스 객체에 들어있는 속성에 대한 접근 관리

객체, 클래스 모두 동일한 이름의 속성>> 객체 통해 속성 접근 시 객체속성이 클래스 속성 가림

### 19.3.2. 프로퍼티 문서화

콘솔의 help() 함수/ IDE 도구가 프로퍼티에 대한 문서 보어주어야 >> 프로퍼티 __doc__ 속성에서 정보 가져옴
  1. weight=propety(get_weight, set_weight, doc='weight in kilograms') : doc 인수로 전달된 문자열 받음. 고전적인 호출구문 사용.
  2. 프로퍼터를 데커레이터로 사용 : @property 데커레이터로 장식된 게터 메서드의 문서화 문자열이 프로퍼티 전체의 문서로 사용됨.

## 19.4. 프로퍼티 팩토리 구현하기

예) quantity() : 프로퍼티 팩토리. 이 팩토리가 관리하는 속성은 0보다 큰 값만을 가져야 함.

weight=quantity('weight') : quantity() 호출 시 weight 클래스 속성은 존재하지 않음

## 19.5. 속성 제거 처리하기

del 문 이용 >> 객체 속성 제거 가능

프로퍼티로 속성을 제거하는 연산도 지원 됨.
  @my_propety.deleter 로 장식된 데커리터 : 이 프로퍼티에 의해 관리되는 속성 제거하는 책임 가짐

## 19.6. 속성을 처리하는 핵심 속성 및 함수

### 19.6.1. 속성 처리에 영향을 주는 특별 속성

__class__ : 객체의 클래스에 대한 참조. 파이썬은 __getattr__() 같은 특별 메서드를 객체 자체가 아니라 객체의 클래스에서만 검색함.
__dict__ : 객체나 클래스의 쓰기가능 속성을 저장하는 매핑. __dict__ 를 가진 객체는 임의의 새로운 속성을 언제든지 설정할 수 있음. 클래스에 __slots__ 속성이 있으면 이 클래스 객체에는 __dict__ 가 없을 수도
__slots__ : 자신의 객체가 가질 수 있는 속성을 제한하려는 클래스에 정의하는 속성. __slots__ 는 허용된 속성명을 담은 일종의 튜플임. __dict__ 가 __slots__ 에 들어 있지 않으면 이 클래스 객체는 자체적인 __dict__ 를 가질 수 없고, 여기에 나열된 속성만 만들 수 있음.

### 19.6.2. 속성을 처리하는 내장 함수

dir([object]) : 대부분의 객체 속성을 나열함. dir()은 대화형 세션에서 사용하기 위한 것으로 전체 속성의 리스트를 제공하지 않지만 '흥미로운' 속성명들을 나열함. 객체에 __dict__ 가 있든 없든 dir()은 객체 내부 조사 가능. __dict__ 자체는 dir()이 나열x. but __dict__ 에 들어있는 키들은 나열. __mro__(), __bases__(), __name__() 등의 특별 메서드도 나열x. object는 선택적 인수로 지정 x 시 현재 범위에 있는 이름들을 나열함.
getattr(object,name[, default]) : object에서 name 문자열로 식별된 속성을 가져옴. 객체의 클래스나 슈퍼클래스에서 속성 가져올 수 있음. 이러한 속성 존재x >> getatttr()은 AttributeError 발생/ default 값 반환.
hasattr(object, name) : 해당 이름의 속성이 object에 있거나 상속등의 메커니즘으로 가져올 수 있으면 True를 반환. 이 메서드는 getattr(object, name)을 호출하고 AttributeError 예외 발생하는지 아닌지 확인해 구현함.
setattr(object, name, value) : object가 허용하면 name 속성에 value를 할당. 새로운 속성 생성되거나 기존 속성의 값 변경됨.
vars([object]) : object의 __dict__ 를 반환. __slots__ 는 있고, __dict__ 는 없는 클래스의 객체는 처리할 수 없음. 인수 전달x >> 현재 범위의 __dict__ 가져옴 >> locals()와 동일하게 작동.

### 19.6.3. 속성을 처리하는 특별 메서드

__delattr__(self,name) : del ㅁㄴ을 이용해 속성 제거하려 할 때 호출됨.
__dir__(self) : 속성을 나열하기 위해 객체에 dir() 호출할 때 호출됨.
__getatt__(self,name) : obj, Class, Class의 슈퍼클래스를 검색해서 명명된 속성을 가져오려고 시도하다 실패할 때 호출됨.
__getattribute__(self, name) : 특별 속성이나 메서드가 아닌 속성을 가져올 때 언제나 호출됨. 점 표기법 및 getattr(), hasattr() 가 이 메서드 호출함.
__setattr__(self, name, value) : 지명된 속성에 값을 설정할 때 언제나 호출 됨. 점 표기법, setattr()가 이 메서드 호출함.

# 20. 속성 디스크립터

디스크립터 이용 >> 여러 속성에 대한 동일한 접근 논리를 재사용할 수 있음.

디스크립터 : __get__(), __set__(), __delete__() 메서드로 구성된 프로토콜을 구현하는 클래스

## 20.1. 디스크립터 예 : 속성 검증

### 20.1.1. LineItem 버전3) 간단한 디스크립터

디스크립터 클래스 : 디스크립터 프로토콜을 구현하는 클래스
관리 대상 클래스 : 디스크립터 객체를 클래스 속성으로 선언하는 클래스
디스크립터 객체 : 관리 대상 클래스의 클래스 속성으로 선언된 디스크립터 클래스의 객체
관리 대상 객체 : 관리 대상 클래스의 객체
저장소 속성 : 관리 대상 객체 안의 관리 대상 속성값을 담을 속성
관리 대상 속성 : 디스크립터 객체에 의해 관리되는 관리 대상 클래스 안의 공개 속성. 속성의 값은 저장소 속성에 저장됨.

### 20.1.2. 버전4) 자동 저장소 속석명

Quantity 객체의 storage_name에 대한 고유한 문자열 생성 >> 속성명 반복 입력 x

__get__() 메서드는 self, instane, owner 등 3개의 인수를 받음. owner 인수는 ㅗ간리 대상 클래스에 대한 참조이며 디스크립터를 이용해 클래스 속성 가져올 때 유용하게 사용할 수 있음. owner에 None>..

### 20.1.3. 버전5) 새로운 디스크립터형

Quantity : 관리 대상 객체의 저장소 속성을 관리, 이 속성 설정 위해 사용되는 값 검증 
2개의 베이스 클래스생성 >> 확장 용이
 AutoStorage : 저장소 속성을 자동으로 관리하는 디스크립터 클래스
 Validated : __set__() 메서드를 오버라이드 해 서브클래스에서 반드시 구현해야하는 validate() 메서드를 호츨하는 AutoStorage의 추상 서브클래스

템플릿 메서드는 서브클래스가 구체적인 동작을 구현하기 위해 오버라이드하는 추상적인 연산의 관점에서 알고리즘을 정의한다.

## 20.2. 오버라이딩 디스크립터와 논오버라이딩 디스크립터

객체 통해 속성 읽음 >> 객체에 정의된 속성 반환
객체 통해 속성 읽음 >> 객체에 그 속성 x >> 클래스 속성 읽음

__set__() 메서드의 정의 여부에 따라 2가지 범주의 디스크립터를 생성함.

### 20.2.1. 오버라이딩 디스크립터

__set__() 메서드 구현하는 디스크립터 : 오버라이딩 디스크립터
오버라이딩 디스크림터 : 객체 속성에 할당하려는 시도 가로챔. 프로퍼티도 오버라이딩 디스크립터

프로퍼티에 세터 함수 제공x 해도 __set__()가 읽기 전용 속성인거 알려주려고 AttributeError 발생함.

### 20.2.2. __get__()이 없는 오버라이딩 디스크립터

__set__ 메서드만 오버라이드, __get__ x >> 저장 연산만 디스크립터가 처리

읽기 접근 처리하는 __get__() 없음>> 객체 통해 디스크립터 확인시 디스크립터 객체 자체가 반환됨.
쓰기 접근은 __set__()가 가로채지만, 속성 읽을 때는 디스크립터 객체가 아니라 새로운 값을 그대로 반환. 즉, 읽기 연산의 경우만 객체 속성이 디스크립터 가림.

### 20.2.3. 논오버라이딩 디스크립터

__set__() 메서드 구현x 한 디스크립터. 메서드.

### 20.2.4. 클래스 안에서 디스크립터 덮어쓰기

클래스의 속성에 값 할당 >> 덮어써짐.

## 20.3. 메서드는 디스크립터

모든 사용자 정의 함수 : __get__() 메서드 가지고 있음. >> 클래스 연결 함수는 디스크립터로 작동 >> 클래스 안 함수는 클래스에 바인딩된 메서드가 됨.

함수가 __set__() 메서드 구현 x 이므로 함수는 논오버라이딩 디스크립터임. 

## 20.4. 디스크립터 사용에 대한 조언

코드는 간결하게 작성하기 위해 프로퍼티 사용.
읽기 전용 디스크립터는 __set__()을 구현해야 함
검증 디스크립터는 __set__()만 사용할 수 있음
캐시는 __get__()에서만 효율적으로 구현할 수 있음
특별 메서드 이외의 메서드는 객체 속성에 의해 가려질 수 있음

## 20.5. 디스크립터의 문서화 문자열과 관리 대상 속성의 삭제

디스크립터 클래스의 문서화 문자열 : 관리 대상 클래스에 있는 모든 디스크립터 객체를 문서화하기 위해 사용됨



